# Charlson Comorbidity Index - Python

This calculation of the Charlson Comorbidity Index (CCI) has been designed to work on the OMOP CDM v5.3 and developed in the University of North Carolina at Chapel Hill de-identified OMOP Research Data Repository, ORDR(D). Please see the README in the repo with important notes, clarifications, and assumptions.


Author: Josh Fuchs

Copyright 2025, The University of North Carolina at Chapel Hill. Permission is granted to use in accordance with the MIT license.
The code is licensed under the open-source MIT license.

# Load packages, connect to the database, define functions

In [1]:
import sqlalchemy
import psycopg
import pandas as pd
import numpy as np

from getpass import getpass
from itertools import groupby

In [2]:
# prompt for username and password
user_name = input('enter username:')
user_password = getpass('enter password:')


## configure connection
conn = psycopg.connect(
    host='od2-primary',
    dbname='ordrd',
    user=user_name,
    password=user_password)

enter username: jtfuchs
enter password: ········


In [3]:
cursor = conn.cursor()

In [4]:
def query_to_dataframe(query_string, column_names):
    """
    Runs a SQL query on the database and returns the results
    as a pandas DataFrame.
    
    PARAMETERS:
    query_string : string
        string formatted as a SQL command, to execute on the database
        
    column_name : list
        list of strings for desired names of returned columns
        length must match the number of columsn returned in the
        query        
        
        If you are selecting all (*) columns from a single table,
        this value can be None. In that case, this function will
        query the table to get the column names and use those to
        create the dataframe
    
    RETURNS:
        conditional. If error with query, returns
        
        query_result : None
        
        if query is successful, returns
        
        query_df : pandas dataframe
            DataFrame with column names set from column_names
    """
    try:
        cursor.execute(query_string)
        query_result = cursor.fetchall()
    except:
        print("Error with SQL command")
        conn.rollback()
        query_result = None    

    if query_result is not None:
        if column_names is None:
            # first, get the table name
            table_name = extract_substring(query_string).strip()
            # build the query to get the column names
            column_name_query = "select column_name from information_schema.columns where table_name = '{}';".format(table_name)
            # execute the query
            cursor.execute(column_name_query)
            column_names_tuple = cursor.fetchall()
            # convert tuples to strings
            column_names = [' '.join(item) for item in column_names_tuple]
        query_df = pd.DataFrame(query_result,columns=column_names)
        return query_df
    
    return query_result

In [5]:
def extract_substring(query_string):
    """
    Takes an input query string and returns the name of 
    the table that is being queried. Only works for a single
    table. 
    
    Example: if you are querying omop.measurement, this 
    will return measurement
    
    PARAMETERS:
    query_string : string
        query string
        
    RETURNS
    table_string : string
        table that is being queried. Found after omop.
    """
    # Find the index of "omop"
    start_index = query_string.find("omop")
    if start_index != -1:
        # Find the index of the next space after "omop"
        end_index = query_string.find(" ", start_index)
        if end_index != -1:
            # Extract the desired substring
            table_string = query_string[start_index + 4:end_index]
            # the substring will begin with a .
            # ignore that
            return table_string[1:]
    return None  # Return None if "omop" or space is not found

In [6]:
def split_list_to_dict(lst):
    """
    Splits a Python list into a dictionary where the keys are the length of each entry
    and the values are the corresponding entries.
    
    PARAMETERS:
    lst : list
        list of strings
        
    RETURNS
    result_dict : dictionary
        dictionary where keys are the length of entries in lst, and values are entries
        in lst that correspond to that length
    """
    result_dict = {}
    for item in lst:
        key = len(item)
        if key not in result_dict:
            result_dict[key] = []
        result_dict[key].append(item)
    return result_dict

In [7]:
def add_new_row(df):
    """
    Adds a new row to the dataframe if the condition_source_value is 4373. This is to account
    for 437.3 and 437 being listed as two sepate indicators in Quan et al. (2005). 
    
    For patients with this condition, this function appends a row so that both formats
    are included. 
    
    PARAMETERS
    df : pandas DataFrame
    
    RETURNS
    new_df : pandas DataFrame
    new rows are appended at the bottom of the input dataframe
    """
    # Identify rows where condition_source_value is '4373'
    mask = df['condition_source_value'] == '4373'
    
    # Create a new DataFrame with the same person_id and condition_source_value = '437'
    new_row = df[mask].copy()
    new_row['condition_source_value'] = '437'
    
    # Append the new row to the original DataFrame
    new_df = pd.concat([df,new_row],ignore_index=True)
    
    return new_df

In [8]:
def calculate_score(condition_list,weight_col):
    """
    Calculates the CCI given a list of conditions a patient has. If a condition appears twice in the condition_df,
    then it will be counted twice here. 
      
    PARAMETERS
    condition_list : list
        list of conditions a patient has diagnoses for
        
    weight_col : string
        column in condition_df containing the weights for each condition group
        
    RETURNS
    total_score : int
        CCI for these conditions
    """
    total_score = 0
    
    # iterate through the condition_df to go through,
    # condition by condition, and see if any values in the
    # condition list match any of the values for that particular
    # condition. If so, sum the score for that condition to 
    # the overall score
    for index, row in condition_df.iterrows():
        if bool(set(condition_list).intersection(set(row['icd']))):
            total_score += row[weight_col]
    return total_score

# Build DataFrame for Conditions

We'll build a pandas DataFrame that will store ICD codes and weights for each Charlson condition. 

First, create a dictionary with conditions, ICD codes, and weights.

ICD codes: These are both ICD-9 and ICD-10 codes from Quan et al. (2005). These have been stripped of periods
to make matching easier in the database. 


In [9]:

columns = ['icd','charlson86_weight']

condition_dictionary = {'myocardial infarction' : [['410','412','I21','I22','I252'],1],
                        'congestive heart failure' : [['39891','40201','40211','40291','40401','40403','40411','40413','40491','40493',
                                                       '4254','4255','4256','4257','4258','4259','428','I099','I110','I130','I132',
                                                       'I255','I420','I425','I426','I427','I428','I429','P290','I43','I50'],1],
                        'peripheral vascular disease' : [['0930','4373','4431','4432','4433','4434','4435','4436','4437','4438','4439',
                                                          '4471','5571','5579','V434','440','441','I731','I738','I739','I771','I790',
                                                          'I792','K551','K558','K559','Z958','Z959','I70','I71'],1],
                        'cerebrovascular disease' : [['430','431','432','433','434','435','436','437','438','36234','G45','G46','I60',
                                                      'I61','I62','I63','I64','I65','I66','I67','I68','I69','H340'],1],
                        'dementia' : [['29410','29411','3312','290','F051','G311','F00','F01','F02','F03','G30'],1],
                        'chronic pulmonary disease' : [['490','491','492','493','494','495','496','500','501','502','503','504','505',
                                                        '4168','4169','5064','5081','5088','J40','J41','J42','J43','J44','J45','J46',
                                                        'J47','J60','J61','J62','J63','J64','J65','J66','J67','I278','I279','J684',
                                                        'J701','J703'],1],
                        'connective tissue disease' : [['4465','7100','7101','7102','7103','7104','7140','7141','7142','7148','725','M315',
                                                        'M351','M353','M360','M05','M32','M33','M34','M06'],1],
                        'ulcer disease' : [['531','532','533','534','K25','K26','K27','K28'],1],
                        'mild liver disease' : [['07022','07023','07032','07033','07044','07054','0706','0709','5733','5734','5738',
                                                 '5739','V427','570','571','K700','K701','K702','K703','K709','K717','K713','K714',
                                                 'K715','K760','K762','K763','K764','K768','K769','Z944','B18','K73','K74'],1],
                        'diabetes without complications' : [['2500','2501','2502','2503','2508','2509','E100','E101','E106','E108',
                                                             'E109','E110','E111','E116','E118','E119','E120','E121','E126','E128',
                                                             'E129','E130','E131','E136','E138','E139','E140','E141','E146','E148',
                                                             'E149'],1],
                        'hemiplegia' : [['3341','3440','3441','3442','3443','3444','3445','3446','3449','342','343','G041','G114',
                                         'G801','G802','G830','G831','G832','G833','G834','G839','G81','G82'],2],
                        'renal disease' : [['40301','40311','40391','40402','40403','40412','40413','40492','40493','5830','5831',
                                            '5832','5834','5836','5837','5880','V420','V451','582','585','586','V56','N18','N19',
                                            'N052','N053','N054','N055','N056','N057','N250','I120','I131','N032','N033','N034',
                                            'N035','N036','N037','Z490','Z491','Z492','Z940','Z992'],2],
                        'diabetes with complications' : [['2504','2505','2506','2507','E102','E103','E104','E105','E107','E112',
                                                          'E113','E114','E115','E117','E122','E123','E124','E125','E127','E132',
                                                          'E133','E134','E135','E137','E142','E143','E144','E145','E147'],2],
                        'cancer' : [['140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155',
                                     '156','157','158','159','160','161','162','163','164','165','170','171','172','174','175','176',
                                     '179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194',
                                     '195','200','201','202','203','204','205','206','207','208','2386','C00','C01','C02','C03','C04',
                                     'C05','C06','C07','C08','C09','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20',
                                     'C21','C22','C23','C24','C25','C26','C30','C31','C32','C33','C34','C37','C38','C39','C40','C41',
                                     'C43','C45','C46','C47','C48','C49','C50','C51','C52','C53','C54','C55','C56','C57','C58','C60',
                                     'C61','C62','C63','C64','C65','C66','C67','C68','C69','C70','C71','C72','C73','C74','C75','C76',
                                     'C81','C82','C83','C84','C85','C88','C90','C91','C92','C93','C94','C95','C96','C97'],2],
                        'moderate or severe liver disease' : [['4560','4561','4562','5722','5723','5724','5728','K704','K711','K721',
                                                               'K729','K765','K766','K767','I850','I859','I864','I982'],3],
                        'metastatic cancer' : [['196','197','198','199','C77','C78','C79','C80'],6],
                        'aids' : [['042','043','044','B20','B21','B22','B24'],6]
}



In [10]:
condition_df = pd.DataFrame.from_dict(condition_dictionary,orient='index',columns=columns)
condition_df.head()

,icd,charlson86_weight
myocardial infarction,"[410, 412, I21, I22, I252]",1
congestive heart failure,"[39891, 40201, 40211, 40291, 40401, 40403, 404...",1
peripheral vascular disease,"[0930, 4373, 4431, 4432, 4433, 4434, 4435, 443...",1
cerebrovascular disease,"[430, 431, 432, 433, 434, 435, 436, 437, 438, ...",1
dementia,"[29410, 29411, 3312, 290, F051, G311, F00, F01...",1


# Extract the ICD codes and reformat for querying

Pull out the ICD codes from the DataFrame, combine into a single list, then split into a dictionary by length of code. 


In [11]:
# extract ICD codes
condition_list_icd = condition_df['icd'].to_list()

# flatten list
condition_list_icd = [item for sublist in condition_list_icd for item in sublist]

# split list into dictionary by length of code
condition_dictionary_icd = split_list_to_dict(condition_list_icd)


## Query Database 

Next, we'll query the database using the ICD codes, returning a DataFrame for analysis. Logic:

1) First, join bith dates for each person to condition_occurrence table, then only keep conditions that are (1) from the problem list and (2) when condition start date is on or after the birth date

2) from this joined table, select rows where the condition_source_value matches any of the condition ICD codes and return those codes for downstream analysis

This query takes all recorded diagnoses across the linespan of a patient. If you want to filter the database by a global date range, please see the next subsection.

In [12]:
condition_query = ("""\
                    with condition_start_filter as (
                    select  vco.*
                    from omop.v_condition_occurrence as vco
                    left join omop.person as p
                    on vco.person_id = p.person_id
                    where condition_type_concept_id = 32840
                    and (vco.condition_start_date - p.birth_datetime::date) >= 0
                    )    
                    Select DISTINCT person_id, 
                    CASE WHEN substring(translate(condition_source_value,'.',''),1,4) in ('4373') then '4373' 
                    WHEN substring(condition_source_value,1,3) in {0} then substring(condition_source_value,1,3)
                    WHEN substring(translate(condition_source_value,'.',''),1,4) in {1} then substring(translate(condition_source_value,'.',''),1,4)
                    WHEN substring(translate(condition_source_value,'.',''),1,5) in {2} then substring(translate(condition_source_value,'.',''),1,5)
                    ELSE NULL END AS condition_source_value,
                    condition_source_concept_vocabulary_id
                    FROM condition_start_filter
                    WHERE CASE
                    WHEN substring(translate(condition_source_value,'.',''),1,4) in ('4373') then  1
                    WHEN substring(condition_source_value,1,3) in {0} then 1
                    when substring(translate(condition_source_value,'.',''),1,4) in {1} then 1 
                    when substring(translate(condition_source_value,'.',''),1,5) in {2} then 1 
                    ELSE 0
                     END = 1;
                    """).format(tuple(condition_dictionary_icd[3]),tuple(condition_dictionary_icd[4]),tuple(condition_dictionary_icd[5]))


In [13]:

column_names = ['person_id','condition_source_value','condition_source_concept_vocabulary_id']
person_condition_df = query_to_dataframe(condition_query,column_names)


### Query With Date

This query operates identically to the one above, but is restricted to a date range provides in the query. The dates are specified in lines 9 and 10 of the query:

```
and vco.condition_start_date >= '2015-01-03'  
and vco.condition_start_date < '2016-01-03' 
```

You can change the dates at the end of both of lines to set the range that you want to filter on. Dates must be formatted as YYYY-MM-DD. This filters the entire database. You should run this query instead of the one below, then continue with the notebook. 

In [28]:
condition_query_with_daterange = ("""\
                    with condition_start_filter as (
                    select  vco.*
                    from omop.v_condition_occurrence as vco
                    left join omop.person as p
                    on vco.person_id = p.person_id
                    where condition_type_concept_id = 32840
                    and (vco.condition_start_date - p.birth_datetime::date) >= 0
                    and vco.condition_start_date >= '2015-01-03'  
                    and vco.condition_start_date < '2016-01-03' 
                    )    
                    Select DISTINCT person_id, 
                    CASE WHEN substring(translate(condition_source_value,'.',''),1,4) in ('4373') then '4373' 
                    WHEN substring(condition_source_value,1,3) in {0} then substring(condition_source_value,1,3)
                    WHEN substring(translate(condition_source_value,'.',''),1,4) in {1} then substring(translate(condition_source_value,'.',''),1,4)
                    WHEN substring(translate(condition_source_value,'.',''),1,5) in {2} then substring(translate(condition_source_value,'.',''),1,5)
                    ELSE NULL END AS condition_source_value,
                    condition_source_concept_vocabulary_id
                    FROM condition_start_filter
                    WHERE CASE
                    WHEN substring(translate(condition_source_value,'.',''),1,4) in ('4373') then  1
                    WHEN substring(condition_source_value,1,3) in {0} then 1
                    when substring(translate(condition_source_value,'.',''),1,4) in {1} then 1 
                    when substring(translate(condition_source_value,'.',''),1,5) in {2} then 1 
                    ELSE 0
                     END = 1;
                    """).format(tuple(condition_dictionary_icd[3]),tuple(condition_dictionary_icd[4]),tuple(condition_dictionary_icd[5]))


In [30]:
column_names_with_daterange = ['person_id','condition_source_value','condition_source_concept_vocabulary_id']
person_condition_df = query_to_dataframe(condition_query_with_daterange,column_names_with_daterange)


In [32]:
person_condition_df.shape

(152357, 3)

# For condition 437.3, we need to list this twice in the person_condition_df

Once as 4373 and once as 437. We don't need to do this for the other duplicated ICD-9 codes because the ```calculate_score``` function deals with those correctly because it is the same code listed twice, not slightly differently. 

In [14]:
person_condition_df = add_new_row(person_condition_df)

## Check that subset of conditions correspond to ICD9, not ICD10

Check for the following conditions: V43.4, V42.7, V42.0, V45.1, V56.x

These can correspond to codes in both ICD9 and ICD10, but for Charlson should be ICD9 codes (per Quan 2005). Therefore, remove rows where the condition is one of those and the source vocabulary is ICD10CM. 

In [15]:
# Filter out rows where condition_source is V42 or V44 and vocabulary_id is ICD10CM
person_condition_df = person_condition_df[~((person_condition_df['condition_source_value'].isin(['V434', 'V427', 'V420','V451','V56'])) & 
                   (person_condition_df['condition_source_concept_vocabulary_id'] == 'ICD10CM'))]



# Group conditions by person_id

In [16]:
# use groupby to get a list of all conditions for each person
person_condition_grouped_df = person_condition_df.groupby('person_id')['condition_source_value'].apply(list).reset_index()


# Calculate CCI based on conditions

In [17]:
# specify column name for condition weights in the condition_df
weight_column = 'charlson86_weight'

person_condition_grouped_df['comorbidity_score'] = person_condition_grouped_df.apply(
    lambda row: calculate_score(row['condition_source_value'],weight_column),axis=1)


In [18]:
person_condition_grouped_df.shape

(789974, 3)

In [19]:
#person_condition_grouped_df.to_csv('charlson_python.csv')